In [232]:
import numpy as np
import pandas as pd
import optuna
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

In [233]:
df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
df

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595207,1488013,0,3,1,10,0,0,0,0,0,...,4,1,9,6,0,1,1,0,1,1
595208,1488016,0,5,1,3,0,0,0,0,0,...,4,1,3,8,1,0,1,0,1,1
595209,1488017,0,1,1,10,0,0,1,0,0,...,3,2,2,6,0,0,1,0,0,0
595210,1488021,0,5,2,3,1,0,0,0,1,...,4,1,4,2,0,1,1,1,0,0


In [234]:
print(len(df))
print(len(test_df))

595212
892816


In [214]:
n_estimators = 1500
early_stopping_round = 150
cv_folds = 5
random_state = 0
params = {#'objective': 'binary',
          #    'boosting_type': 'gbdt',
              'learning_rate': 0.01,
              'max_bin': 25,
              'num_leaves': 31,
              'min_child_samples': 1500,
              'colsample_bytree': 0.7,
              'subsample_freq': 1,
              'subsample': 0.7,
              'reg_alpha': 1.0,
              'reg_lambda': 1.0,
              'verbosity': 0,
              'random_state': 0}

In [215]:
# remove calc features
calc_features = [feat for feat in df.columns if "_calc" in feat]
df = df.drop(calc_features, axis="columns")
test_df = test_df.drop(calc_features, axis="columns")

In [216]:
# one-hot encoding
cat_features = [feat for feat in df.columns if "_cat" in feat]
df = pd.get_dummies(df, columns=cat_features)
test_df = pd.get_dummies(test_df, columns=cat_features)

In [217]:
# target encoding
#for column in cat_features:
#    value_dict = dict(df.groupby([column])['target'].mean())
#    df[column] = df[column].apply(lambda x: value_dict.get(x))
#    test_df[column] = test_df[column].apply(lambda x: value_dict.get(x))

In [218]:
X_columns = df.columns.drop(['id', 'target'])
y_column = ['target']

In [219]:
from numba import jit

@jit
def eval_gini(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_pred)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

def gini_lgb(y_true, y_pred):
    eval_name = 'normalized_gini_coef'
    eval_result = eval_gini(y_true, y_pred)
    is_higher_better = True
    return eval_name, eval_result, is_higher_better

/var/folders/st/0pg24vg97s98zyv3mbmdmzbc0000gn/T/ipykernel_55789/2566664522.py:4: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def eval_gini(y_true, y_pred):


In [220]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
train_data, test_data = train_test_split(df, stratify=df['target'], test_size=0.3, random_state=100)

In [221]:
# 损失函数
#def micro_f1_scorer(y, y_pred):
#    return f1_score(y, y_pred, average='micro')

def gini(actual, pred):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float64)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

gini_normalized_score = make_scorer(gini_normalized)

In [222]:
regressor_oof = pd.Series(len(df))
regressor_preds = np.zeros(len(test_df))
metric_evaluations = list()

skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=random_state)

for idx, (train_idx, valid_idx) in enumerate(skf.split(df[X_columns], df[y_column])):
    print(f"CV fold {idx}")
    X_train, y_train = df[X_columns].iloc[train_idx], df[y_column].iloc[train_idx]
    X_valid, y_valid = df[X_columns].iloc[valid_idx], df[y_column].iloc[valid_idx]
    
    #model = lgb.LGBMClassifier(**params,
    #                           n_estimators=n_estimators,
    #                           early_stopping_round=early_stopping_round,
    #                           force_row_wise=True)
    model = lgb.LGBMRegressor(**params,
                               n_estimators=n_estimators,
                               early_stopping_round=early_stopping_round,
                               force_row_wise=True)
    
    callbacks=[lgb.early_stopping(stopping_rounds=150), 
               lgb.log_evaluation(period=100, show_stdv=False)]
                                                                                           
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric=gini_lgb, callbacks=callbacks)
    metric_evaluations.append(model.best_score_['valid_0']['normalized_gini_coef'])
    regressor_oof.iloc[valid_idx] = model.predict(X_valid, num_iteration=model.best_iteration_)
    #regressor_preds += model.predict_proba(test_df[X_columns], num_iteration=model.best_iteration_)[:,1] / skf.n_splits
    regressor_preds = model.predict(test_df[X_columns], num_iteration=model.best_iteration_) / skf.n_splits

CV fold 0
Training until validation scores don't improve for 150 rounds
[100]	valid_0's l2: 0.0348691	valid_0's normalized_gini_coef: 0.270577
[200]	valid_0's l2: 0.0347868	valid_0's normalized_gini_coef: 0.279449
[300]	valid_0's l2: 0.0347498	valid_0's normalized_gini_coef: 0.285956
[400]	valid_0's l2: 0.0347312	valid_0's normalized_gini_coef: 0.289761
[500]	valid_0's l2: 0.0347219	valid_0's normalized_gini_coef: 0.291894
[600]	valid_0's l2: 0.0347154	valid_0's normalized_gini_coef: 0.293434
[700]	valid_0's l2: 0.0347119	valid_0's normalized_gini_coef: 0.293871
[800]	valid_0's l2: 0.0347095	valid_0's normalized_gini_coef: 0.294577
[900]	valid_0's l2: 0.0347096	valid_0's normalized_gini_coef: 0.294127
CV fold 1
Training until validation scores don't improve for 150 rounds
[100]	valid_0's l2: 0.0348931	valid_0's normalized_gini_coef: 0.255967
[200]	valid_0's l2: 0.0348286	valid_0's normalized_gini_coef: 0.261662
[300]	valid_0's l2: 0.0348076	valid_0's normalized_gini_coef: 0.265066
[400

In [223]:
print(f"LightGBM CV Gini Normalized Score: {np.mean(metric_evaluations):0.5f} ({np.std(metric_evaluations):0.5f})")

LightGBM CV Gini Normalized Score: 0.28773 (0.01534)


In [224]:
submission = pd.DataFrame()
submission['id'] = test_df['id']
submission['target'] = regressor_preds
submission.to_csv('lgb_regressor_dumpies_remove_calc.csv', index=False, sep=',')

In [229]:
params = {#'objective': 'binary',
          #    'boosting_type': 'gbdt',
              'learning_rate': 0.01,
              'max_bin': 25,
              'num_leaves': 31,
              'min_child_samples': 1500,
              'colsample_bytree': 0.7,
              'subsample_freq': 1,
              'subsample': 0.7,
              'reg_alpha': 1.0,
              'reg_lambda': 1.0,
              'verbosity': 0,
              'random_state': 0}

classifier_oof = pd.Series(np.zeros(len(df)))
classifier_preds = np.zeros(len(test_df))
metric_evaluations = list()

skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=random_state)

for idx, (train_idx, valid_idx) in enumerate(skf.split(df[X_columns], df[y_column])):
    print(f"CV fold {idx}")
    X_train, y_train = df[X_columns].iloc[train_idx], df[y_column].iloc[train_idx]
    X_valid, y_valid = df[X_columns].iloc[valid_idx], df[y_column].iloc[valid_idx]
    
    #model = lgb.LGBMClassifier(**params,
    #                           n_estimators=n_estimators,
    #                           early_stopping_round=early_stopping_round,
    #                           force_row_wise=True)
    model = lgb.LGBMClassifier(**params,
                               n_estimators=n_estimators,
                               early_stopping_round=early_stopping_round,
                               force_row_wise=True)
    
    callbacks=[lgb.early_stopping(stopping_rounds=150), 
               lgb.log_evaluation(period=100, show_stdv=False)]
                                                                                           
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric=gini_lgb, callbacks=callbacks)
    metric_evaluations.append(model.best_score_['valid_0']['normalized_gini_coef'])
    classifier_oof.iloc[valid_idx] = model.predict_proba(X_valid, num_iteration=model.best_iteration_)[:,1]
    classifier_preds += model.predict_proba(test_df[X_columns], num_iteration=model.best_iteration_)[:,1] / skf.n_splits
    #classifier_preds = model.predict(test_df[X_columns], num_iteration=model.best_iteration_) / skf.n_splits

CV fold 0


/Users/clair/opt/anaconda3/envs/py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/clair/opt/anaconda3/envs/py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Users/clair/opt/anaconda3/envs/py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.153243	valid_0's normalized_gini_coef: 0.271457
[200]	valid_0's binary_logloss: 0.15228	valid_0's normalized_gini_coef: 0.280599
[300]	valid_0's binary_logloss: 0.15185	valid_0's normalized_gini_coef: 0.286829
[400]	valid_0's binary_logloss: 0.151651	valid_0's normalized_gini_coef: 0.289906
[500]	valid_0's binary_logloss: 0.151543	valid_0's normalized_gini_coef: 0.291906
[600]	valid_0's binary_logloss: 0.151473	valid_0's normalized_gini_coef: 0.293377
[700]	valid_0's binary_logloss: 0.151437	valid_0's normalized_gini_coef: 0.293827
[800]	valid_0's binary_logloss: 0.151417	valid_0's normalized_gini_coef: 0.294276
[900]	valid_0's binary_logloss: 0.15142	valid_0's normalized_gini_coef: 0.294119
Early stopping, best iteration is:
[806]	valid_0's binary_logloss: 0.151416	valid_0's normalized_gini_coef: 0.294311
CV fold 1


/Users/clair/opt/anaconda3/envs/py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/clair/opt/anaconda3/envs/py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Users/clair/opt/anaconda3/envs/py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.153553	valid_0's normalized_gini_coef: 0.255568
[200]	valid_0's binary_logloss: 0.152779	valid_0's normalized_gini_coef: 0.261176
[300]	valid_0's binary_logloss: 0.152509	valid_0's normalized_gini_coef: 0.264598
[400]	valid_0's binary_logloss: 0.152392	valid_0's normalized_gini_coef: 0.266942
[500]	valid_0's binary_logloss: 0.152334	valid_0's normalized_gini_coef: 0.268508
[600]	valid_0's binary_logloss: 0.15231	valid_0's normalized_gini_coef: 0.269259
[700]	valid_0's binary_logloss: 0.152308	valid_0's normalized_gini_coef: 0.269299
[800]	valid_0's binary_logloss: 0.1523	valid_0's normalized_gini_coef: 0.269814
[900]	valid_0's binary_logloss: 0.152298	valid_0's normalized_gini_coef: 0.270119
CV fold 2


...

Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.15349	valid_0's normalized_gini_coef: 0.250439
[200]	valid_0's binary_logloss: 0.152638	valid_0's normalized_gini_coef: 0.261464
[300]	valid_0's binary_logloss: 0.152286	valid_0's normalized_gini_coef: 0.267762
[400]	valid_0's binary_logloss: 0.15211	valid_0's normalized_gini_coef: 0.271645
[500]	valid_0's binary_logloss: 0.152011	valid_0's normalized_gini_coef: 0.274328
[600]	valid_0's binary_logloss: 0.151963	valid_0's normalized_gini_coef: 0.275614
[700]	valid_0's binary_logloss: 0.151928	valid_0's normalized_gini_coef: 0.276735
[800]	valid_0's binary_logloss: 0.151915	valid_0's normalized_gini_coef: 0.277063
[900]	valid_0's binary_logloss: 0.151899	valid_0's normalized_gini_coef: 0.277667
[1000]	valid_0's binary_logloss: 0.151906	valid_0's normalized_gini_coef: 0.277539
Early stopping, best iteration is:
[934]	valid_0's binary_logloss: 0.151897	valid_0's normalized_gini_coef: 0.277743
CV

/Users/clair/opt/anaconda3/envs/py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/clair/opt/anaconda3/envs/py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Users/clair/opt/anaconda3/envs/py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.153151	valid_0's normalized_gini_coef: 0.28492
[200]	valid_0's binary_logloss: 0.152081	valid_0's normalized_gini_coef: 0.294826
[300]	valid_0's binary_logloss: 0.151594	valid_0's normalized_gini_coef: 0.301155
[400]	valid_0's binary_logloss: 0.151332	valid_0's normalized_gini_coef: 0.305416
[500]	valid_0's binary_logloss: 0.151173	valid_0's normalized_gini_coef: 0.308713
[600]	valid_0's binary_logloss: 0.151074	valid_0's normalized_gini_coef: 0.310518
[700]	valid_0's binary_logloss: 0.151014	valid_0's normalized_gini_coef: 0.311803
[800]	valid_0's binary_logloss: 0.150976	valid_0's normalized_gini_coef: 0.312533
[900]	valid_0's binary_logloss: 0.150947	valid_0's normalized_gini_coef: 0.31291
[1000]	valid_0's binary_logloss: 0.150928	valid_0's normalized_gini_coef: 0.313239
[1100]	valid_0's binary_logloss: 0.15092	valid_0's normalized_gini_coef: 0.313284
[1200]	valid_0's binary_logloss: 0.15

/Users/clair/opt/anaconda3/envs/py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/clair/opt/anaconda3/envs/py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Users/clair/opt/anaconda3/envs/py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.153417	valid_0's normalized_gini_coef: 0.259211
[200]	valid_0's binary_logloss: 0.152528	valid_0's normalized_gini_coef: 0.268649
[300]	valid_0's binary_logloss: 0.152123	valid_0's normalized_gini_coef: 0.27534
[400]	valid_0's binary_logloss: 0.15192	valid_0's normalized_gini_coef: 0.279392
[500]	valid_0's binary_logloss: 0.151813	valid_0's normalized_gini_coef: 0.28184
[600]	valid_0's binary_logloss: 0.151733	valid_0's normalized_gini_coef: 0.283741
[700]	valid_0's binary_logloss: 0.151682	valid_0's normalized_gini_coef: 0.284936
[800]	valid_0's binary_logloss: 0.151647	valid_0's normalized_gini_coef: 0.286006
[900]	valid_0's binary_logloss: 0.151625	valid_0's normalized_gini_coef: 0.28652
[1000]	valid_0's binary_logloss: 0.151608	valid_0's normalized_gini_coef: 0.286903
[1100]	valid_0's binary_logloss: 0.151595	valid_0's normalized_gini_coef: 0.287265
[1200]	valid_0's binary_logloss: 0.151

In [230]:
print(f"LightGBM CV Gini Normalized Score: {np.mean(metric_evaluations):0.5f} ({np.std(metric_evaluations):0.5f})")

LightGBM CV Gini Normalized Score: 0.28859 (0.01495)


In [231]:
submission = pd.DataFrame()
submission['id'] = test_df['id']
submission['target'] = classifier_preds
submission.to_csv('lgb_classifier_dumpies_remove_calc.csv', index=False, sep=',')

In [ ]:
# average ensemble
submit_data = pd.DataFrame()
submit_data['id'] = test_df['id']
submit_data['target'] = [(regressor_preds[i] + classifier_preds[i])/2 for i in range(len(test_df))]
submit_data.to_csv('lgb_average_ensemble_best_params_dumpies_remove_calc.csv', index=False, sep=',')

In [ ]:
# weight ensemble
best_score = 0
best_weight = 0
for w in range(0,101):
    weight = w/100
    oof = [(regressor_oof[i]*weight + classifier_oof[i]*(1-weight)) for i in range(len(df))]
    score = gini_normalized(df['target'], oof)
    print("weight: %s, score: %s" % (weight, score))
    if score > best_score:
        best_score = score
        best_weight = weight

print("best weight: %s, best score: %s" % (best_weight, best_score))

In [ ]:
submit_data = pd.DataFrame()
submit_data['id'] = test_df['id']
submit_data['target'] = [(regressor_preds[i]*best_weight + classifier_preds[i]*(1-best_weight)) for i in range(len(test_df))]
submit_data.to_csv('lgb_weight_ensemble_my_best_params_dumpies_te_remove_calc.csv', index=False, sep=',')